In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time

from collections import Counter

import pandas as pd
from collections import defaultdict

from python_graphql_client import GraphqlClient
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import csv


import http.client
import shutil
import json
import requests
import os
import numpy as np
import tqdm

from covalent import Client
import asyncio
import scipy.stats
import csv
from itertools import islice
import chardet

### 1. Using Snapshot API, Download proposal data list

In [ ]:
# 1) Get available dao list from Boardroom & snapshot_checked
# Using Boardroom API : Graphql on the boardrrom website

A = pd.read_excel(r"C:\Users\Administrator\Desktop\chunghyun\python\boardroom_snapshot_added.xlsx", sheet_name='over_20proposals')
snapshot_space_list = A['snapshot_name'].dropna().tolist()
snapshot_space_list.remove('-')
snapshot_space_list = set(snapshot_space_list)

snapshot_space_list= list(snapshot_space_list)
snapshot_space_list.sort()
print(len(snapshot_space_list))
snapshot_space_list

In [ ]:
# 2) Using Snapshot API, Download proposal list using grphql
client = Client(transport=RequestsHTTPTransport(url="https://hub.snapshot.org/graphql"))

query = gql("""
    query GetProposals($spaceName: String) {
        proposals(
            first: 1000,
            skip: 0,
            where: { space_in: [$spaceName], state: "closed" },
            orderBy: "created",
            orderDirection: asc
        ) {
            ...proposalFields
        }
    }

    fragment proposalFields on Proposal {
        id
        network
        title
        author
        votes
        type
        choices
        scores
        scores_total
        discussion
        link
        quorum
        start
        end
        state
        space {
            id
            name
        }
    }
""")

'''
proposal_ID : id
network : 1(Ethereum Mainnet), 2(Testnet)
votes : votes count

'''


'\nproposal_ID : id \nnetwork : 1(Ethereum Mainnet), 2(Testnet)\nvotes : votes count\n\n\n'

In [ ]:
## graphql에서 protocol list 구한 후에 for loop로 proposal csv 만들기 by protocol
## add real time column
errored_space = []

for index, space in enumerate(snapshot_space_list[1:]):
    print(f'Processing space {space}, index {index + 1}/{len(snapshot_space_list)}')

    try:
        result = client.execute(query, variable_values={"spaceName": space})
        df = pd.DataFrame(result['proposals'])
        df.insert(13, 'start_real_time', pd.to_datetime(df['start'], unit='s'))
        df.insert(15, 'end_real_time', pd.to_datetime(df['end'], unit='s'))

        file_name = f'C:/Users/Administrator/Desktop/chunghyun/python/proposals_snapshot/{space}_proposals_snapshot.csv'
        df.to_csv(file_name, index=False)

    except Exception as e:
        print(f"Error processing space {space}: {e}")
        errored_space.append(space)

### 2. Downloading voting data by protocol

In [ ]:
# 3) Download all the voting data available in Snapshot using selenium
# 3-1) Download all the voting data available in Snapshot using selenium by protocol

directory_path = r'C:\Users\Administrator\Desktop\chunghyun\python\proposals_snapshot'

# Check if the directory exists
if os.path.exists(directory_path):
    # List all files in the directory
    files = os.listdir(directory_path)
print(len(files))
print(files)


In [ ]:
# 3-2) Checking tarhet item's index in the list
target_item = "tracer.eth" +"_proposals_snapshot.csv"

# Check if the target item is in the list
if target_item in files:
    # Get the index of the target item and print it
    index_of_item = files.index(target_item)
    print(f"The index of {target_item} is: {index_of_item}")
else:
    print(f"{target_item} is not in the list.")

In [ ]:
# 3-3) Downloading voting data using selenium, Sanpshot downloading one by one

for idx, file in enumerate(files[:]):
    print(f'Processing space {file}, index {idx+1}/{len(files[146:155])}')
    csv_file = f"C:/Users/Administrator/Desktop/chunghyun/python/proposals_snapshot/{file}"
    df = pd.read_csv(csv_file)

    # Extracting Proposal ID list
    proposal_ID_list = df['id'].tolist()

    # URL setting : snapshot.org/#/uniswap/proposal/"proposal ID"
    URL_list = []
    space_name = file.split("_")[0]

    base_url = f"https://snapshot.org/#/{space_name}/proposal/"
    for proposal_id in proposal_ID_list:
        url = base_url + proposal_id
        URL_list.append(url)

    # Set the download directory path
    new_folder = f"{space_name}_voting_data"
    download_dir = r"C:\Users\Administrator\Desktop\chunghyun\python\voting_data_by_protocol"
    new_directory_path = os.path.join(download_dir, new_folder)
    os.makedirs(new_directory_path, exist_ok=True)

    # Set the maximum time to wait for an element (in seconds)
    implicit_wait_time = 7

    # Configure ChromeOptions to set the download directory
    options = Options()
    prefs = {"download.default_directory": new_directory_path}
    options.add_experimental_option("prefs", prefs)

    # Start a new Chrome browser instance with the configured options
    driver = webdriver.Chrome(options=options)

    errored_url = []
    try:
        # Set the implicit wait
        driver.implicitly_wait(implicit_wait_time)

        for file_idx, url in enumerate(URL_list):
            try:
                # Open the URL in the browser
                driver.get(url)
                time.sleep(3)
                driver.execute_script("window.scrollBy(0, 300);")
                # Find and click the download button

                download_button_xpath = '//*[@id="content-left"]/div[3]/div/div[1]/button'
                download_button = driver.find_element(By.XPATH, download_button_xpath)

                time.sleep(5)
                download_button.click()

                # Wait for the file to download (you can add an explicit wait here if needed)
                # Here, we use a short sleep as a simple wait for demonstration purposes.
                time.sleep(10)  # Adjust the wait time as needed

                # Get a list of files in the download directory
                downloaded_files = os.listdir(new_directory_path)
                id = url.split("/")[-1]

                # Find the downloaded file name (assuming it is the only file downloaded in the directory)
                downloaded_file_name = None
                for file_name in downloaded_files:
                    if "snapshot-report-" + id in file_name:
                        downloaded_file_name = file_name
                        break

                # Check if the expected file name exists
                if downloaded_file_name is not None:
                    print(f"File {file_idx+1}/{len(URL_list)} downloaded successfully.")
                else:
                    # If the first button fails, try the second button
                    try:
                        cancel_pop_up_button_xpath ='/html/body/div[2]/div/div[2]/button'
                        cancel_pop_up_button = driver.find_element(By.XPATH, cancel_pop_up_button_xpath)
                        cancel_pop_up_button.click()

                        time.sleep(3)
                        driver.execute_script("window.scrollBy(0, 300);")
                        time.sleep(8)

                        second_download_button_xpath = '/html/body/div[1]/div[1]/div[2]/div[3]/div/div/div/div[1]/div[3]/div[2]/div[1]/button'
                        second_download_button = driver.find_element(By.XPATH, second_download_button_xpath)
                        time.sleep(5)
                        second_download_button.click()

                        time.sleep(7)

                        downloaded_files = os.listdir(new_directory_path)
                        downloaded_file_name = None
                        for file_name in downloaded_files:
                            if "snapshot-report-" + id in file_name:
                                downloaded_file_name = file_name
                                break

                        if downloaded_file_name is not None:
                            print(f"File {file_idx+1}/{len(URL_list)} downloaded successfully after second attempt.")
                        else:
                            raise Exception(f"File download failed for URL: {url}.")

                    except Exception as second_button_error:
                        errored_url.append(url)
                        print(f"Error while processing URL: {url} after trying the second button")

            except Exception as e:
                errored_url.append(url)
                print(f"Error while processing URL: {url}.")

        print(f"{space_name}'s errored url len : {len(errored_url)}")
        errored_df = pd.DataFrame({'Errored URL': errored_url})

        # Write errored URLs to a CSV file if there are any
        if not errored_df.empty:
            csv_file_path = f"C:/Users/Administrator/Desktop/chunghyun/python/errored_urls_by_protocol/{space_name}_errored_urls_{len(errored_url)}.csv"
            errored_df.to_csv(csv_file_path, index=False)

            print(f"Errored URLs written to {csv_file_path}.")

        else :
            print("No errors while processing URLs.")

    except Exception as e:
        print(f"Error : error at opening browser")

    finally:
        driver.quit()


### 3. Errored files retrial

In [ ]:
csv_folder = r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han"

# Get a list of CSV files in the folder
csv_files = [file for file in os.listdir(csv_folder) if file.endswith('.csv')]

# Specify the folder containing CSV files
csv_folder = r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han"

# Get a list of CSV files in the folder
csv_files = [file for file in os.listdir(csv_folder) if file.endswith('.csv')]

# 9:13
for file in csv_files[39:]:
    filename = file.split('_')[0]
    print(f"Processing file: {filename}")

    # Read the CSV file and extract the 'errored_url' column
    csv_path = os.path.join(csv_folder, file)
    df = pd.read_csv(csv_path)

    if 'Errored URL' not in df.columns:
        print("No 'Errored URL' column found in the file. Skipping.")
        continue

    errored_url_list = df['Errored URL'].tolist()
    errored_url_list.sort()


    # Set the download directory path
    new_folder = f"{filename}_errored_voting_data_trial2"
    download_dir = r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han\errorred_urls_by_protocol"
    new_directory_path = os.path.join(download_dir, new_folder)
    os.makedirs(new_directory_path, exist_ok=True)

    # Set the maximum time to wait for an element (in seconds)
    implicit_wait_time = 7

    # Configure ChromeOptions to set the download directory
    options = Options()
    prefs = {"download.default_directory": new_directory_path}
    options.add_experimental_option("prefs", prefs)

    # Start a new Chrome browser instance with the configured options
    driver = webdriver.Chrome(options=options)

    errored_url_after_trial2 = []

    try:
        # Set the implicit wait
        driver.implicitly_wait(implicit_wait_time)
        #
        for file_idx, url in enumerate(errored_url_list[:]):
            try:
                # Open the URL in the browser
                driver.get(url)

                # Find and click the download button
                download_button_xpath = '//*[@id="content-left"]/div[3]/div/div[1]/button'
                download_button = driver.find_element(By.XPATH, download_button_xpath)
                time.sleep(1)
                download_button.click()

                # Wait for the file to download (you can add an explicit wait here if needed)
                # Here, we use a short sleep as a simple wait for demonstration purposes.
                time.sleep(5)  # Adjust the wait time as needed

                # Get a list of files in the download directory
                downloaded_files = os.listdir(new_directory_path)
                id = url.split("/")[-1]

                # Find the downloaded file name (assuming it is the only file downloaded in the directory)
                downloaded_file_name = None
                for file_name in downloaded_files:
                    if "snapshot-report-" + id in file_name:
                        downloaded_file_name = file_name
                        break

                # Check if the expected file name exists
                if downloaded_file_name is not None:
                    print(f"File {file_idx+1}/{len(errored_url_list)} downloaded successfully.")
                else:
                    # If the first button fails, try the second button
                    try:
                        cancel_pop_up_button_xpath ='/html/body/div[2]/div/div[2]/button'
                        cancel_pop_up_button = driver.find_element(By.XPATH, cancel_pop_up_button_xpath)
                        cancel_pop_up_button.click()

                        time.sleep(2)
                        driver.execute_script("window.scrollBy(0, 500);")
                        time.sleep(3)

                        second_download_button_xpath = '/html/body/div[1]/div[1]/div[2]/div[3]/div/div/div/div[1]/div[3]/div[2]/div[1]/button'
                        second_download_button = driver.find_element(By.XPATH, second_download_button_xpath)
                        time.sleep(5)
                        second_download_button.click()

                        time.sleep(5)

                        downloaded_files = os.listdir(new_directory_path)
                        downloaded_file_name = None
                        for file_name in downloaded_files:
                            if "snapshot-report-" + id in file_name:
                                downloaded_file_name = file_name
                                break

                        if downloaded_file_name is not None:
                            print(f"File {file_idx+1}/{len(errored_url_list)} downloaded successfully after second attempt.")
                        else:
                            raise Exception(f"File download failed for URL: {url}.")

                    except Exception as second_button_error:
                        errored_url_after_trial2.append(url)
                        print(f"Error while processing URL: {url} after trying the second button")

            except Exception as e:
                errored_url_after_trial2.append(url)
                print(f"Error while processing URL: {url}.")

        print(f"{filename}'s errored url len : {len(errored_url_after_trial2)}")
        errored_df = pd.DataFrame({'Errored URL': errored_url_after_trial2})

        # Write errored URLs to a CSV file if there are any
        if not errored_df.empty:
            csv_file_path = f"C:/Users/ChunghyunHan/Desktop/Chunghyun Han/errorred_urls_trial2/{filename}_errored_urls_{len(errored_url_after_trial2)}_trial2.csv"
            errored_df.to_csv(csv_file_path, index=False)

            print(f"Errored URLs after trial 2 written to {csv_file_path}.")

        else :
            print("No errors while processing URLs.")

    except Exception as e:
        print(f"Error : error at opening browser")

    finally:
        driver.quit()


In [ ]:
df_retrial = pd.read_csv(r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han\errorred_urls_trial2\aavegotchi.eth_errored_urls_61_trial2.csv")
gotchi_retrial = df_retrial['Errored URL'].tolist()

In [ ]:
## files matching test

df_retrial_aave = pd.read_csv(r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han\gnosis.eth_errored_urls_72.csv")
errored_list = df_retrial_aave['Errored URL'].tolist()

errored_trial2 = [url.split("/")[-1] for url in errored_list]
print(len(errored_trial2))
print(errored_trial2)

directory_path = r"C:\Users\ChunghyunHan\Desktop\Chunghyun Han\errorred_urls_by_protocol\gnosis.eth_errored_voting_data_trial2"

# Get a list of files in the directory
files_list = os.listdir(directory_path)
downloaded_proposal_list = [file.split('-')[-1].replace('.csv', '') for file in files_list]

set_errored_aave_trial2 = set(errored_trial2)
set_downloaded_proposal_list = set(downloaded_proposal_list)

# Find missing elements in the downloaded_proposal_list compared to errored_aave_trial2
missing_elements = set_errored_aave_trial2 - set_downloaded_proposal_list
print(len(missing_elements))

print("Missing Elements:")
for element in missing_elements:
    print(element)

### 4. Boardroom DATA Fetching

In [ ]:
# boardroom protocols list
df0 = pd.read_excel(r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\boardroom_snapshot_added_1124.xlsx", sheet_name='over_20proposals')
boardroom_list = df0['cname'].dropna().tolist()

In [6]:
# Function to fetch protocol information for a given protocol
def fetch_protocol_data(protocol):
    conn = http.client.HTTPSConnection("api.boardroom.info")
    headers = {'Accept': "application/json"}

    # Concatenate the API key and protocol to the end of the URL
    url = f"/v1/protocols/{protocol}?excludeTokenInfo=false&includeContractMetadata=true&limit=3500&status=closed&key={api_key}"

    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    return json.loads(res.read())['data']

api_key = '6e171f4a120b41da20910cc813ddf6a0'
# https://api.boardroom.info/v1/protocols/{cname}

In [ ]:
protocol_data_list = []

for protocol in boardroom_list:
    protocol_data = fetch_protocol_data(protocol)
    protocol_data_list.append(protocol_data)

protocols_df = pd.DataFrame(protocol_data_list)

protocols_df.head()
protocols_df.to_csv(r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\boardroom_protocols_data.csv", index=False)

In [44]:
boardroom_protocols = protocols_df['cname'].tolist()
boardroom_protocols.sort()
print(len(boardroom_protocols))
print(boardroom_protocols[166:168])
# error : 'magicappstoreeth', stargatefi 

190
['stargatefi', 'streamreth']


In [45]:
conn = http.client.HTTPSConnection("api.boardroom.info")

def fetch_snapshot_data(protocol, limit=3500):
    headers = {'Accept': "application/json"}

    # Concatenate the API key, protocol, limit, and other parameters to the end of the URL
    url = f"/v1/protocols/{protocol}/proposals?orderByIndexedAt=asc&limit={limit}&key={api_key}"

    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    return json.loads(res.read())['data']

# Directory to save CSV files
save_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Boardroom&Snapshot_data\Boardroom_proposal_data"

# Function to save DataFrame to CSV file
def save_to_csv(df, protocol, directory):
    file_path = f"{directory}\\{protocol}_proposal_data.csv"
    df.to_csv(file_path, index=False)
    

# Loop through each protocol in boardroom_protocols
for idx, protocol in enumerate(boardroom_protocols[167:]):
    snapshot_data = fetch_snapshot_data(protocol)
    
    # Create a DataFrame from the snapshot data
    df_snapshot = pd.DataFrame(snapshot_data)
    df_snapshot = df_snapshot.sort_values(by='startTimestamp')
    # Save the DataFrame to a CSV file using the function
    save_to_csv(df_snapshot, protocol, save_directory)
    print(f"Snapshot data for {protocol} saved, index {idx+1}/{len(boardroom_protocols)}")

# Close the connection
conn.close()

Snapshot data for streamreth saved, index 1/190
Snapshot data for sushi saved, index 2/190
Snapshot data for thedreamdaoeth saved, index 3/190
Snapshot data for thelanddaopropeth saved, index 4/190
Snapshot data for threshold saved, index 5/190
Snapshot data for tigervcdaoeth saved, index 6/190
Snapshot data for token-engineering-commons saved, index 7/190
Snapshot data for tokenlon saved, index 8/190
Snapshot data for tornadocash saved, index 9/190
Snapshot data for tracereth saved, index 10/190
Snapshot data for truefigov saved, index 11/190
Snapshot data for ultradaov2 saved, index 12/190
Snapshot data for uniswap saved, index 13/190
Snapshot data for unlock saved, index 14/190
Snapshot data for vareneth saved, index 15/190
Snapshot data for voteairswapeth saved, index 16/190
Snapshot data for votevitadaoeth saved, index 17/190
Snapshot data for vsp saved, index 18/190
Snapshot data for wagdieeth saved, index 19/190
Snapshot data for wagumi saved, index 20/190
Snapshot data for xdai

### 5. Merged_data Building for each protocol

In [ ]:
# Win rate for each proposal of all protocols 

    # check the columns are identical for the protocols
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\proposals_snapshot_valid'

# Ideal column list
ideal_columns = ['id', 'network', 'title', 'author', 'votes', 'type', 'choices', 'scores', 'scores_total', 'discussion', 'link', 'quorum', 'start', 'start_real_time', 'end', 'end_real_time', 'state', 'space']

# Function to check if columns match the ideal list
def check_columns(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']

    try:
        # Read the CSV file with detected encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Get the columns of the DataFrame
        columns = list(df.columns)

        # Check if columns match the ideal list
        if columns != ideal_columns:
            print(f"Columns in {os.path.basename(file_path)} do not match the ideal list.")
            print(f"Ideal columns: {ideal_columns}")
            print(f"Actual columns: {columns}")
            print()

        else :
            print(f"Columns in {os.path.basename(file_path)} match the ideal list.")
    except UnicodeDecodeError:
        print(f"UnicodeDecodeError: Unable to read file {os.path.basename(file_path)} with detected encoding '{encoding}'. Trying with 'utf-8'.")
        
        # Retry reading the CSV file with 'utf-8' encoding
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
            
            # Get the columns of the DataFrame
            columns = list(df.columns)

            # Check if columns match the ideal list
            if columns != ideal_columns:
                print(f"Columns in {os.path.basename(file_path)} do not match the ideal list.111111")
                print(f"Ideal columns: {ideal_columns}")
                print(f"Actual columns: {columns}")
                print()
        except pd.errors.EmptyDataError:
            print(f"File {os.path.basename(file_path)} is empty.")
            print()
        except UnicodeDecodeError:
            print(f"Unable to read file {os.path.basename(file_path)} even with 'utf-8' encoding.")
            print()

# Iterate through files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    
    # Check only for CSV files
    if filename.endswith(".csv"):
        check_columns(file_path)

In [ ]:
# Win rate for each proposal of all protocols
# Directory containing the files
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\proposals_snapshot_valid'

# Output directory for modified files
output_directory = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\proposals_snapshot_col_adds'

# Function to process each file
def process_file(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, encoding='ISO-8859-1')

        # Check if 'scores_total' is 0 and drop the corresponding rows
        df = df[df['scores_total'] != 0]

        # Calculate additional columns
        df['Num_choices'] = df['choices'].apply(lambda x: len(eval(x)) if x else 0)
        df['Win_choice'] = df.apply(lambda x: eval(x['choices'])[eval(x['scores']).index(max(eval(x['scores'])))] if x['choices'] and x['scores'] else None, axis=1)
        df['Win_score'] = df['scores'].apply(lambda x: max(eval(x)) if x else None)
        df['Winrate'] = df.apply(lambda row: row['Win_score'] / sum(eval(row['scores'])) if row['scores'] else None, axis=1)

        # Reorder columns to place the new ones between 'scores' and 'scores_total'
        new_columns = ['Num_choices', 'Win_choice', 'Win_score', 'Winrate']
        position_to_insert = df.columns.get_loc('scores_total')  # Get the position of 'scores_total'
        df = pd.concat([df.iloc[:, :position_to_insert], df[new_columns], df.iloc[:, position_to_insert:]], axis=1)

        df = df.iloc[:, :-4]
        
        print(df.head())

        # Construct the output file name
        base_filename, file_extension = os.path.splitext(os.path.basename(file_path))
        output_filename = f"{base_filename}_cols_added{file_extension}"

        # Save the modified DataFrame to a new CSV file
        output_file_path = os.path.join(output_directory, output_filename)
        df.to_csv(output_file_path, index=False)

        print(f"Success: File {output_filename} processed and saved.")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {e}")

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through files in the directory
for filename in os.listdir(directory_path):
    print(filename)
    file_path = os.path.join(directory_path, filename)

    # Check only for CSV files
    if filename.endswith(".csv"):
        process_file(file_path)
        
        

In [ ]:
# Specify the main directory containing subdirectories with CSV files
main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# Get a list of subdirectories (each subdirectory represents a protocol)
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Iterate through each subdirectory (protocol)
for subdir in subdirectories[1:]:
    # Construct the full path to the subdirectory
    subdirectory_path = os.path.join(main_directory, subdir)

    # Get a list of CSV files in the subdirectory
    csv_files = [file for file in os.listdir(subdirectory_path) if file.endswith('.csv')]

    # Create an empty list to store errored CSV files
    errored_files = []

    # Iterate through each CSV file in the subdirectory
    for file in csv_files:
        # Extract proposal_id from the file name
        proposal_id = file.split('-')[2].split('.')[0]

        # Construct the full file path
        file_path = os.path.join(subdirectory_path, file)

        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Rename the 'address' column to 'voter_address'
            df.rename(columns={'address': 'voter_address'}, inplace=True)

            # Insert the 'proposal_id' column as the first column
            df.insert(0, 'proposal_id', proposal_id)

            # Save the modified DataFrame back to the CSV file
            df.to_csv(file_path, index=False)

        except Exception as e:
            # Append the name of the errored CSV file to the list
            errored_files.append(file)
            print(f"Error processing file '{file}': {e}")

    # Create a subdirectory for errored files within the current protocol's directory
    errored_files_subdir = os.path.join(subdirectory_path, 'errored_files')
    os.makedirs(errored_files_subdir, exist_ok=True)

    # Move the errored CSV files into the 'errored_files' subdirectory
    for errored_file in errored_files:
        source_path = os.path.join(subdirectory_path, errored_file)
        dest_path = os.path.join(errored_files_subdir, errored_file)
        shutil.move(source_path, dest_path)

    # Print a message indicating the completion of processing for the current subdirectory
    print(f"Processed folder: {subdir}")

    # Print errored files for the current subdirectory
    if errored_files:
        print(f"Errored files in {subdir}: {errored_files}")

In [ ]:
## RETRIAL 
'''
main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# Get a list of subdirectories (each subdirectory represents a protocol)
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Iterate through each subdirectory (protocol)
for subdir in subdirectories[1:]:
    # Construct the full path to the subdirectory
    subdirectory_path = os.path.join(main_directory, subdir)

    # Get a list of CSV files in the 'errored_files' subdirectory
    errored_files_subdir = os.path.join(subdirectory_path, 'errored_files')
    csv_files = [file for file in os.listdir(errored_files_subdir) if file.endswith('.csv')]

    # Create an empty list to store errored CSV files
    errored_files = []

    # Iterate through each CSV file in the 'errored_files' subdirectory
    for file in csv_files:
        # Extract proposal_id from the file name
        proposal_id = file.split('-')[2].split('.')[0]

        # Construct the full file path
        file_path = os.path.join(errored_files_subdir, file)

        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Rename the 'address' column to 'voter_address'
            df.rename(columns={'address': 'voter_address'}, inplace=True)

            # Insert the 'proposal_id' column as the first column
            df.insert(0, 'proposal_id', proposal_id)

            # Save the modified DataFrame back to the CSV file
            df.to_csv(file_path, index=False)

        except Exception as e:
            # Append the name of the errored CSV file to the list
            errored_files.append(file)
            print(f"Error processing file '{file}' in {subdir}: {e}")

    # Print a message indicating the completion of processing for the current subdirectory
    print(f"Processed folder: {subdir}")

    # Print errored files for the current subdirectory
    if errored_files:
        print(f"Errored files in {subdir}: {errored_files}")
'''
# checking
'''
main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# Get a list of subdirectories (each subdirectory represents a protocol)
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Check if all 'errored_files' folders are empty
all_errored_files_empty = True

# Iterate through each subdirectory (protocol)
for subdir in subdirectories[1:]:
    # Construct the full path to the 'errored_files' subdirectory
    errored_files_subdir = os.path.join(main_directory, subdir, 'errored_files')

    # Check if the 'errored_files' subdirectory is empty
    if os.listdir(errored_files_subdir):
        all_errored_files_empty = False
        print(f"The 'errored_files' folder in {subdir} is not empty.")

# Print the result
if all_errored_files_empty:
    print("All 'errored_files' folders are empty.")
else:
    print("Not all 'errored_files' folders are empty.")
'''

In [ ]:
# # keep only necessary columns
# # Specify the main directory containing subdirectories with CSV files
# main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# # Get a list of subdirectories (each subdirectory represents a protocol)
# subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# # Iterate through each subdirectory (protocol)
# for subdir in subdirectories:
#     # Construct the full path to the subdirectory
#     subdirectory_path = os.path.join(main_directory, subdir)

#     # Get a list of CSV files in the subdirectory
#     csv_files = [file for file in os.listdir(subdirectory_path) if file.endswith('.csv')]

#     # Iterate through each CSV file in the subdirectory
#     for file in csv_files:
#         # Construct the full file path
#         file_path = os.path.join(subdirectory_path, file)

#         # Read the CSV file into a DataFrame
#         df = pd.read_csv(file_path)

#         # Check if the last column is 'timestamp'
#         if df.columns[-1] == 'timestamp':
#             # No further action needed if the last column is already 'timestamp'
#             print(f"File '{file}' already conforms to the desired format.")
#         else:
#             # Drop columns after 'timestamp' and save the modified DataFrame
#             df = df.loc[:, :'timestamp']
#             df.to_csv(file_path, index=False)
#             print(f"Modified file '{file}' to ensure the last column is 'timestamp'.")

#     print(f"Processing completed for subdirectory: {subdir}")

# print("All processing completed.")

In [ ]:
# format uniformity
# Specify the main directory containing subdirectories with CSV files
main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# Get a list of subdirectories (each subdirectory represents a protocol)
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Iterate through each subdirectory (protocol)
for subdir in subdirectories:
    # Construct the full path to the subdirectory
    subdirectory_path = os.path.join(main_directory, subdir)

    # Get a list of CSV files in the subdirectory
    csv_files = [file for file in os.listdir(subdirectory_path) if file.endswith('.csv')]

    # Initialize the count of errored files
    errored_files_count = 0

    # Check if there are CSV files with columns starting with 'choice.'
    if any(any(col.startswith('choice.') for col in pd.read_csv(os.path.join(subdirectory_path, file), nrows=0).columns) for file in csv_files):
        # Create a new folder 'multiple_choices' in the current subdirectory
        multiple_choices_folder = os.path.join(subdirectory_path, 'multiple_choices')
        os.makedirs(multiple_choices_folder, exist_ok=True)

        # Iterate through each CSV file in the subdirectory
        for file in csv_files:
            # Construct the full file path
            file_path = os.path.join(subdirectory_path, file)

            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path, encoding='ISO-8859-1')
                
                # Check if the file has multiple choice columns
                if any(col.startswith('choice.') for col in df.columns):
                    # Move the original file to the 'multiple_choices' folder
                    shutil.move(file_path, os.path.join(multiple_choices_folder, file))
            except Exception as e:
                print(f"Error processing file '{file}': {e}")
                errored_files_count += 1
                # Create a new folder 'errored_files' in the current subdirectory
                errored_files_folder = os.path.join(subdirectory_path, 'errored_files')
                os.makedirs(errored_files_folder, exist_ok=True)
                # Move the errored file to the 'errored_files' folder
                shutil.move(file_path, os.path.join(errored_files_folder, file))

        print(f"Processed folder: {subdir} - 'multiple_choices' folder created.")
        print(f"Number of errored files: {errored_files_count}")
    else:
        print(f"No files with 'choice.' columns found in folder: {subdir}")


In [ ]:
## adding proposal_id for all protocols

import os
import pandas as pd
main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

errored_files = []
# Iterate through subdirectories
for subdir, _, files in os.walk(main_directory):
    for file in files:
        file_path = os.path.join(subdir, file)
        
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path, encoding='ISO-8859-1')

            # Check if 'proposal_id' column already exists
            if 'proposal_id' not in df.columns:
                # Extract proposal_id from the filename
                proposal_id = file.split('-')[2].split('.')[0]
                
                # Add 'proposal_id' column with the extracted proposal_id
                df.insert(0, 'proposal_id', proposal_id)

                # Save the modified DataFrame back to the original file
                df.to_csv(file_path, index=False)
                print(f"Added 'proposal_id' column to file: {file} in directory: {subdir}")
            else:
                # print(f"'proposal_id' column already exists in file: {file} in directory: {subdir}")
                pass
        except Exception as e:
            print(f"Error processing file '{file}' in directory: {subdir} - {e}")
            errored_files.append(file_path)

print(f"Number of errored files: {len(errored_files)}")

### merged files building
1) preprocess : make the form uniformly, and concat (stacked), drop 'reason' column
2) merge inner, with cols_added file and boardroom contents file
3) merge 2) file to the 'xxx_concat file' 
    (1) cols_added file column rename : id -> proposal_id
    (2) cols_added file column 'network', 'space' drop
    (3) _concat file column 'author_ipfs_hash' drop
    (4) _concat file 'choice' col is nan -> drop the line


FT, NFT seperately done

In [ ]:
# preprocessing : building concat files
import os
import pandas as pd

main_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\voting_data_by_protocol_master"

# Specify the directory to save merged files
output_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\preprocessing_voting_concat"

# Get a list of protocol folders
protocol_folders = [folder for folder in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, folder))]

for protocol_folder in protocol_folders[70:]:
    print(f"Processing protocol folder: {protocol_folder}")
    # Construct the full path to the protocol folder
    protocol_folder_path = os.path.join(main_directory, protocol_folder)

    # Get a list of CSV files in the protocol folder
    csv_files = [file for file in os.listdir(protocol_folder_path) if file.endswith('.csv')]

    # Create an empty list to store DataFrames
    dfs = []

    # Iterate through each CSV file in the protocol folder
    for csv_file in csv_files:
        # Construct the full file path
        file_path = os.path.join(protocol_folder_path, csv_file)

        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path, low_memory=False)

            # Drop the 'reason' column
            df = df.drop('author_ipfs_hash', axis=1, errors='ignore')
            df = df.drop('reason', axis=1, errors='ignore')

            # Append the DataFrame to the list
            dfs.append(df)
        except Exception as e:
            print(f"Error in file: {file_path}. Error: {e}")
            
    # Concatenate all DataFrames into one vertically
    concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)

    # Save the concatenated DataFrame to a new CSV file
    output_file_path = os.path.join(output_directory, f"{protocol_folder}_concat.csv")
    concatenated_df.to_csv(output_file_path, index=False)

    print(f"Processed protocol folder: {protocol_folder}")

print("Merging and preprocessing completed.")

In [89]:
## Merge 1 : cols_added+boardroom data
file1_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\proposals_snapshot_col_adds"
file2_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Boardroom&Snapshot_data\Boardroom_proposal_data_contents"

# Specify the output directory for merged files
output_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\preprocessing111"

def extract_columns(file_path):
    df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
    protocol_name = file_path.split('_')[0]
    df.rename(columns={'id': 'proposal_id'}, inplace=True)
    df['proposal_id'] = df['proposal_id'].astype(str).str.lower().str.strip()
    return df[['proposal_id', 'blockNumber']], protocol_name

# Get the list of files in both directories
files_dir1 = [f for f in os.listdir(file1_directory) if f.endswith('.csv')] #cols_added
files_dir2 = [f for f in os.listdir(file2_directory) if f.endswith('.csv')] #contents 
# merge 2 to 1

# Iterate through files in directory 1
for file1 in files_dir1[32:33]:
    # Extract protocol name from file name
    protocol_name = file1.split('_')[0].split('.')[0]
    print(protocol_name)
    
    # Find matching file in directory 2
    matching_file_in_dir2 = next((f for f in files_dir2 if f == f"{protocol_name}_proposal_data.csv"), None)

    if matching_file_in_dir2:
        # Extract relevant columns from directory 2
        df_dir2, protocol_name_dir2 = extract_columns(os.path.join(file2_directory, matching_file_in_dir2))

        # Read and extract relevant columns from directory 1
        df_dir1 = pd.read_csv(os.path.join(file1_directory, file1), encoding='ISO-8859-1', low_memory=False)
        df_dir1.rename(columns={'id': 'proposal_id'}, inplace=True)
        df_dir1['proposal_id'] = df_dir1['proposal_id'].astype(str).str.lower().str.strip()
        df_dir1 = df_dir1.drop(['network', 'space'], axis=1, errors='ignore')

        df_dir2['proposal_id'] = df_dir2['proposal_id'].astype(str).str.lower().str.strip()
        df_merged = pd.merge(df_dir1, df_dir2, on='proposal_id', how='inner')

        # Save the merged DataFrame to a new CSV file in the output directory
        output_file_path = os.path.join(output_directory, f"{protocol_name}.eth_preprocessing_intersected.csv")
        df_merged.to_csv(output_file_path, index=False)

        print(f"Merged and saved file for protocol: {protocol_name}")
    else:
        print(f"No matching file found for {file1} in directory 2.")

cream-finance
Merged and saved file for protocol: cream-finance


In [100]:
# Merging files
file1_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\preprocessing_voting_concat\NFT" 
file2_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\preprocessing111\NFT"

# Specify the output directory for merged files
output_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\NFT"

protocols = list(set(file.split('_')[0] for file in os.listdir(file1_directory) if file.endswith('.csv')))
protocols.sort()

for protocol in protocols:
    # File 1
    file1_name = f"{protocol}_voting_data_concat.csv"
    file1_path = os.path.join(file1_directory, file1_name)

    # File 2
    file2_name = f"{protocol}_preprocessing_intersected.csv"
    file2_path = os.path.join(file2_directory, file2_name)

    # Output file
    output_file_name = f"{protocol}_merged_data.csv"
    output_file_path = os.path.join(output_directory, output_file_name)

    # Read file1
    df_file1 = pd.read_csv(file1_path,encoding='ISO-8859-1')
    df_file1.rename(columns={'address': 'voter_address'}, inplace=True, errors='ignore')
    df_file1['proposal_id'] = df_file1['proposal_id'].astype(str).str.lower().str.strip()

    # Drop rows where 'choice' column is empty
    # df_file1 = df_file1.dropna(subset=['choice'])

    # Read file2
    df_file2 = pd.read_csv(file2_path, encoding='ISO-8859-1')
    

    # Rename 'id' column to 'proposal_id' and drop 'network' and 'space' columns from file2
    df_file2.rename(columns={'id': 'proposal_id'}, inplace=True, errors='ignore')
    df_file2['proposal_id'] = df_file2['proposal_id'].astype(str).str.lower().str.strip()
    # df_file2 = df_file2.drop(['network', 'space'], axis=1, errors='ignore')

    # Merge file1 and file2 on 'proposal_id'
    df_merged = pd.merge(df_file1, df_file2, on='proposal_id', how='left')
    df_merged.dropna(subset=['title'], inplace=True)
    df_merged['relative_time_voting'] = (df_merged['timestamp'] - df_merged['start']) / (df_merged['end'] - df_merged['start'])

    # Save the merged DataFrame to a new CSV file
    df_merged.to_csv(output_file_path, index=False)

    print(f"Merged and saved file for protocol: {protocol}")

print("Merging and saving completed.")

Merged and saved file for protocol: brightmoments.eth
Merged and saved file for protocol: bullsontheblock.eth
Merged and saved file for protocol: buzzedbears.eth
Merged and saved file for protocol: citydao.eth
Merged and saved file for protocol: doodles.eth
Merged and saved file for protocol: evmavericks.eth
Merged and saved file for protocol: expansiondao.eth
Merged and saved file for protocol: fatcatsdao.eth
Merged and saved file for protocol: futera.eth
Merged and saved file for protocol: gmdao.eth
Merged and saved file for protocol: gnars.eth
Merged and saved file for protocol: grailers.eth
Merged and saved file for protocol: grayboys.eth
Merged and saved file for protocol: loot-dao.eth
Merged and saved file for protocol: mutantsdao.eth
Merged and saved file for protocol: nation3.eth
Merged and saved file for protocol: thedreamdao.eth
Merged and saved file for protocol: thelanddaoprop.eth
Merging and saving completed.


In [13]:
'''
#####
# filtering_intersection : merge with boardroom 
dir1 = "C:/Users/ChunghyunHan/Desktop/DAO_Voting_1101/merged_data_files/ERC20_BRC20_FT"
# File directory 2
dir2 = "C:/Users/ChunghyunHan/Desktop/DAO_Voting_1101/Boardroom&Snapshot_data/Boardroom_proposal_data_contents"
# Output directory
output_dir = "C:/Users/ChunghyunHan/Desktop/DAO_Voting_1101/merged_data_files/ERC20_BRC20_FT/Merged_Boardroom_Snapshot"

def extract_columns(file_path):
    df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
    protocol_name = file_path.split('_')[0]
    df.rename(columns={'id': 'proposal_id'}, inplace=True)
    df['proposal_id'] = df['proposal_id'].astype(str).str.lower().str.strip()
    return df[['proposal_id', 'blockNumber']], protocol_name


# Get the list of files in both directories
files_dir1 = [f for f in os.listdir(dir1) if f.endswith('.csv')]
files_dir2 = [f for f in os.listdir(dir2) if f.endswith('.csv')]

# Iterate through files in directory 1
for file1 in files_dir1[2:3]:
    # Extract protocol name from file name
    protocol_name = file1.split('_')[0].split('.')[0]
    
    # Find matching file in directory 2
    matching_file2 = next((f for f in files_dir2 if f.startswith(protocol_name)), None)

    if matching_file2:
        # Extract relevant columns from directory 2
        df_dir2, protocol_name_dir2 = extract_columns(os.path.join(dir2, matching_file2))

        # Read and extract relevant columns from directory 1
        df_dir1 = pd.read_csv(os.path.join(dir1, file1), encoding='ISO-8859-1', low_memory=False)
        df_dir2['proposal_id'] = df_dir2['proposal_id'].astype(str).str.lower().str.strip()
        df_merged = pd.merge(df_dir1, df_dir2, on='proposal_id', how='inner')

        # Save the merged DataFrame to a new CSV file in the output directory
        output_file_path = os.path.join(output_dir, f"{protocol_name}.eth_merged_data_intersected.csv")
        df_merged.to_csv(output_file_path, index=False)

        print(f"Merged and saved file for protocol: {protocol_name}")
    else:
        print(f"No matching file found for {file1} in directory 2.")

'''

Merged and saved file for protocol: aave


In [6]:
## accountability column building in merged files

import pandas as pd
import os 

def process_csv_files(directory):
    # List all csv files in the directory
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

    # Process each csv file
    for csv_file in csv_files:
        # Read the csv file
        df = pd.read_csv(os.path.join(directory, csv_file))
        
        # Calculate the accountability for each voter in each proposal
        df['accountability'] = df['voting_power'] / df.groupby('proposal_id')['voting_power'].transform('sum')
        
        # Rank each voter in each proposal by their accountability
        df['voter_rank'] = df.groupby('proposal_id')['accountability'].rank(ascending=False, method='min')
        
        # Count the total number of voters in each proposal
        df['total_voters'] = df.groupby('proposal_id')['voter_address'].transform('nunique')
        
        # Save the modified dataframe back to csv
        df.to_csv(os.path.join(directory, csv_file), index=False)

        print(f"{csv_file} has been updated with new columns and saved back.")

    return 'DONE'

# Replace with the path to your CSV files
directory = r'D:\DAO_Voting_1101\merged_data_files\FT'
process_csv_files(directory)

0xgov.eth_merged_data.csv has been updated with new columns and saved back.
1inch.eth_merged_data.csv has been updated with new columns and saved back.
aave.eth_merged_data.csv has been updated with new columns and saved back.


C:\Users\ChunghyunHan\AppData\Local\Temp\ipykernel_29476\3409468978.py:13: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, csv_file))


aavegotchi.eth_merged_data.csv has been updated with new columns and saved back.
abachi.eth_merged_data.csv has been updated with new columns and saved back.
abracadabrabymerlinthemagician.eth_merged_data.csv has been updated with new columns and saved back.
alchemistcoin.eth_merged_data.csv has been updated with new columns and saved back.
alchemixstakers.eth_merged_data.csv has been updated with new columns and saved back.
alpacafinance.eth_merged_data.csv has been updated with new columns and saved back.
ampleforthorg.eth_merged_data.csv has been updated with new columns and saved back.
apecoin.eth_merged_data.csv has been updated with new columns and saved back.
apwine.eth_merged_data.csv has been updated with new columns and saved back.


C:\Users\ChunghyunHan\AppData\Local\Temp\ipykernel_29476\3409468978.py:13: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, csv_file))


arbitrumfoundation.eth_merged_data.csv has been updated with new columns and saved back.
babydogevote.eth_merged_data.csv has been updated with new columns and saved back.


...

C:\Users\ChunghyunHan\AppData\Local\Temp\ipykernel_29476\3409468978.py:13: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, csv_file))


speraxdao.eth_merged_data.csv has been updated with new columns and saved back.
spiralgov.eth_merged_data.csv has been updated with new columns and saved back.
spookyswap.eth_merged_data.csv has been updated with new columns and saved back.
stakedao.eth_merged_data.csv has been updated with new columns and saved back.
streamr.eth_merged_data.csv has been updated with new columns and saved back.
sushigov.eth_merged_data.csv has been updated with new columns and saved back.
tecommons.eth_merged_data.csv has been updated with new columns and saved back.
theopendao.eth_merged_data.csv has been updated with new columns and saved back.
threshold.eth_merged_data.csv has been updated with new columns and saved back.
tokenlon.eth_merged_data.csv has been updated with new columns and saved back.
tomoondao.eth_merged_data.csv has been updated with new columns and saved back.
tracer.eth_merged_data.csv has been updated with new columns and saved back.
truefigov.eth_merged_data.csv has been updated

C:\Users\ChunghyunHan\AppData\Local\Temp\ipykernel_29476\3409468978.py:13: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, csv_file))


uniswap.eth_merged_data.csv has been updated with new columns and saved back.
unlock-protocol.eth_merged_data.csv has been updated with new columns and saved back.
usebraintrust.eth_merged_data.csv has been updated with new columns and saved back.
varen.eth_merged_data.csv has been updated with new columns and saved back.
vote-airswap.eth_merged_data.csv has been updated with new columns and saved back.
vote-vitadao.eth_merged_data.csv has been updated with new columns and saved back.
vsp.eth_merged_data.csv has been updated with new columns and saved back.
xdaistake.eth_merged_data.csv has been updated with new columns and saved back.
yam.eth_merged_data.csv has been updated with new columns and saved back.
ybaby.eth_merged_data.csv has been updated with new columns and saved back.


'DONE'

### 6. Analysis

### basic stats
1) unique_voters, authors
2) winrate
3) author stats : top 1~3 authors
4) VP_gini coefficient
5) num voters to achieve 50%
6) voter stats : unique voters count
7) participation rate (out of all voter set)
8) top actively participating voters
9) turnover ratio (two consecutive proposals)
10) whale top 5 address, voter accountability, voting timing (other voters' voting timing)

In [33]:
# 1. basic stats
# Input directory containing CSV files
input_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\FT"

# Create an empty list to store the statistics
stats_list = []
# cream, decentraland, rallygov

# Iterate through each CSV file in the input directory
for file_name in os.listdir(input_directory):
    if file_name.endswith('.csv'):
        # Extract protocol_name from the file name
        protocol_name = file_name.split('_')[0].split('.')[0]

        # Read the CSV file into a DataFrame
        file_path = os.path.join(input_directory, file_name)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
        df = df.dropna(subset=['choice'])

        # Count unique values in the 'proposal_id' column
        proposals_count = df['proposal_id'].nunique()
        unique_voters_count = df['voter_address'].nunique()
        unique_authors_count = df['author'].nunique()
       
        # win_rate stats : DF
        grouped_winrate = df.groupby('proposal_id')['Winrate'].agg(['min']) # all the same values for each group-> extract using 'min'
        grouped_winrate.rename(columns={'min': 'winrate_by_proposal'}, inplace=True)

        # NUM_votes stats : Series
        num_votes_valid = df.groupby('proposal_id').size()
        num_votes_valid = num_votes_valid.rename('num_votes_valid').reset_index()


        # Add a new dictionary to the list
        stats_list.append({
            'protocol_name': protocol_name,
            'proposals_count': proposals_count,
            'unique_voters_count': unique_voters_count,  
            'unique_authors_count': unique_authors_count,
            'min_win_rate': grouped_winrate['winrate_by_proposal'].min(),
            '25th_per_win_rate': grouped_winrate['winrate_by_proposal'].quantile(0.25),
            'median_win_rate': grouped_winrate['winrate_by_proposal'].median(),
            '75th_per_win_rate': grouped_winrate['winrate_by_proposal'].quantile(0.75),
            'max_win_rate': grouped_winrate['winrate_by_proposal'].max(),
            'avg_win_rate': grouped_winrate['winrate_by_proposal'].mean(),
            'std_win_rate' : grouped_winrate['winrate_by_proposal'].std(),
            'Sum_valid_votes' : num_votes_valid['num_votes_valid'].sum(),
            'min_num_votes': num_votes_valid['num_votes_valid'].min(),
            '25th_percentile_num_votes': num_votes_valid['num_votes_valid'].quantile(0.25),
            'median_num_votes': num_votes_valid['num_votes_valid'].median(),
            '75th_percentile_num_vote': num_votes_valid['num_votes_valid'].quantile(0.75),
            'max_num_votes': num_votes_valid['num_votes_valid'].max(),
            'avg_num_votes' : num_votes_valid['num_votes_valid'].mean(),
            'std_num_votes' : num_votes_valid['num_votes_valid'].std()
        })

# Create the statistics DataFrame
stats_df = pd.DataFrame(stats_list)

## Save the statistics DataFrame to a CSV file
#output_file_path = os.path.join(output_directory, 'proposals_statistics.csv')
#stats_df.to_csv(output_file_path, index=False)
#print(f"Statistics saved to: {output_file_path}")


In [34]:
# Output directory for the statistics CSV file
output_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Statistics"
output_file_path = os.path.join(output_directory, 'FT_statistics.xlsx')
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Write the first sheet
    stats_df[['protocol_name', 'proposals_count', 'unique_voters_count', 'unique_authors_count', 'min_win_rate', '25th_per_win_rate', 'median_win_rate',
              '75th_per_win_rate', 'max_win_rate', 'avg_win_rate', 'std_win_rate']].to_excel(writer, sheet_name='proposal_voter_authors_winrate', index=False)

    # Write the second sheet
    stats_df[['protocol_name', 'Sum_valid_votes', 'min_num_votes', '25th_percentile_num_votes',
              'median_num_votes', '75th_percentile_num_vote', 'max_num_votes', 'avg_num_votes', 'std_num_votes']].to_excel(writer, sheet_name='votes_stats_by_proposal', index=False)

In [2]:
# 2. author stats
# Directory containing CSV files
directory_path = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\NFT"

# Initialize an empty list to store the statistics
author_stats_list = []

# Initialize lists to store top authors for each protocol
top_author_1_list, top_author_2_list, top_author_3_list = [], [], []

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        try:
            # Extract protocol name from the file name
            protocol_name = filename.split('_')[0]

            # Read the CSV file into a DataFrame
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

            # Convert columns to lowercase and strip whitespace
            df['proposal_id'] = df['proposal_id'].astype(str).str.lower().str.strip()
            df['author'] = df['author'].astype(str).str.lower().str.strip()

            all_authors = []  # Initialize list to store all authors for each protocol

            # Group by 'proposal_id' and create a DataFrame for each group
            for proposal_id, group_df in df.groupby('proposal_id'):
                # Create a new DataFrame with one row for the group
                group_data = {'proposal_id': proposal_id, 'author': group_df['author'].iloc[0]}
                group_stats_df = pd.DataFrame([group_data])

                # Print or append the DataFrame to the list as needed
                authors_list = group_stats_df['author'].tolist()
                all_authors.extend(authors_list)

            author_counts = dict(Counter(all_authors))
            top_authors = [author for author, _ in Counter(all_authors).most_common(3)]

            # Append to the corresponding lists
            top_author_1_list.append(top_authors[0] if len(top_authors) > 0 else None)
            top_author_2_list.append(top_authors[1] if len(top_authors) > 1 else None)
            top_author_3_list.append(top_authors[2] if len(top_authors) > 2 else None)

            # Create a dictionary of author values, their occurrence count, and top authors
            author_stats = {
                'protocol_name': protocol_name,
                'unique_authors': len(author_counts),
                'occur_dict': author_counts,
                'top_author_1': top_author_1_list[-1],  # Use the last value in the list
                'top_1_occurrence': author_counts.get(top_author_1_list[-1], {}),
                'top_author_2': top_author_2_list[-1],
                'top_2_occurrence': author_counts.get(top_author_2_list[-1], {}),
                'top_author_3': top_author_3_list[-1],
                'top_3_occurrence': author_counts.get(top_author_3_list[-1], {})
            }

            # Append the statistics to the list
            author_stats_list.append(author_stats)

        except Exception as e:
            print(f"Error processing file: {file_path}\nError: {str(e)}")

author_stats_df = pd.DataFrame(author_stats_list)
author_stats_df.to_excel(r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Statistics\NFT_author_stats.xlsx", index=False)

In [28]:
import os
import pandas as pd

# Directory containing CSV files
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\FT'

# Initialize an empty dictionary to store the voter and author sets for each protocol
voter_sets_by_protocol = {}
author_sets_by_protocol = {}

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Extract protocol name from the file name
        protocol_name = filename.split('_')[0].split('.')[0]

        # Read the CSV file into a DataFrame
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Get the full voter set for the current protocol (case-insensitive)
        voter_set = set(df['voter_address'].str.lower())
        voter_sets_by_protocol[protocol_name] = voter_set

        # Get the full author set for the current protocol (case-insensitive)
        author_set = set(df['author'].str.lower())
        author_sets_by_protocol[protocol_name] = author_set

# Initialize empty dictionaries to store the intersection between any two protocols
intersection_dict = {}

# Compare the sets for each pair of protocols
protocols = list(voter_sets_by_protocol.keys())
for i in range(len(protocols)):
    for j in range(i + 1, len(protocols)):
        protocol1 = protocols[i]
        protocol2 = protocols[j]

        # Find the intersection between the two sets for 'voter_address' (case-insensitive)
        voter_intersection = voter_sets_by_protocol[protocol1].intersection(voter_sets_by_protocol[protocol2])

        # Find the intersection between the two sets for 'author' (case-insensitive)
        author_intersection = author_sets_by_protocol[protocol1].intersection(author_sets_by_protocol[protocol2])

        # Store the intersections and their lengths in the result dictionary
        intersection_dict[f'{protocol1} - {protocol2}'] = {
            'Protocol 1 Voter Set Length': len(voter_sets_by_protocol[protocol1]),
            'Protocol 2 Voter Set Length': len(voter_sets_by_protocol[protocol2]),
            'Voter Intersection': list(voter_intersection),
            'Voter Intersection Length': len(voter_intersection),
            'Author Intersection': list(author_intersection),
            'Author Intersection Length': len(author_intersection)
        }

# Create a DataFrame from the result dictionary
result_df = pd.DataFrame(intersection_dict).T.reset_index()
result_df.rename(columns={'index': 'Protocol Pair'}, inplace=True)

# Save to a single Excel file
result_df.to_excel(r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Statistics\voter_author_intersection_FT.xlsx", index=False)


In [ ]:
import os 
import pandas as pd
# voter stats

# Specify the directory containing the files
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\FT'

# Initialize an empty list to store the statistics
stats_list = []

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        protocol_name = filename.split("_")[0].split(".")[0]
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        full_voter_set = set(df['voter_address'])

        proposal_voters_dict = {}
        for _, row in df.iterrows():
            proposal_id = row['proposal_id']
            voter_address = row['voter_address']

            if proposal_id not in proposal_voters_dict:
                proposal_voters_dict[proposal_id] = set()

            proposal_voters_dict[proposal_id].add(voter_address)

        participation_rate_dict = {}
        for proposal_id, voters_in_proposal in proposal_voters_dict.items():
            participation_rate = len(voters_in_proposal) / len(full_voter_set)
            participation_rate_dict[proposal_id] = participation_rate

        voter_participation_count = {}
        for voter in full_voter_set:
            count = sum(1 for voters_in_proposal in proposal_voters_dict.values() if voter in voters_in_proposal)
            voter_participation_count[voter] = count

        highest_parti_rate_proposal = max(participation_rate_dict, key=participation_rate_dict.get)
        highest_participation_rate = participation_rate_dict[highest_parti_rate_proposal]
        avg_participation_rate = sum(participation_rate_dict.values()) / len(participation_rate_dict)
        min_participation_rate = min(participation_rate_dict.values())

        top_participating_voters = sorted(voter_participation_count.items(), key=lambda x: x[1], reverse=True)[:5]
        unique_proposal_ids = len(set(df['proposal_id']))


        sort_parti_rate = sorted(participation_rate_dict.items(), key=lambda x: x[1], reverse=True)
        top_50_parti_rate = dict(sort_parti_rate[:50])

        sort_voter_participation_count = sorted(voter_participation_count.items(), key=lambda x: x[1], reverse=True)
        top_50_voter_participation_count = dict(sort_voter_participation_count[:50])

        # Add additional columns to the result dictionary
        result = {
            'protocol_name': protocol_name,
            'full_voter_set_size': len(full_voter_set),
            'unique_proposal_ids': unique_proposal_ids,
            'top_50_participation_rate_dict': top_50_parti_rate,
            'top50_voter_participation_count':top_50_voter_participation_count,
            'highest_participation_proposal': highest_parti_rate_proposal,
            'avg_participation_rate': avg_participation_rate,
            'min_participation_rate': min_participation_rate,
            'highest_participation_rate': highest_participation_rate,
            'top5_participating_voters': [voter for voter, _ in top_participating_voters],
            'top5_participating_counts': [count for _, count in top_participating_voters]
        }

        stats_list.append(result)

# Create a DataFrame from the list of statistics
stats_df = pd.DataFrame(stats_list)

# Print the DataFrame
stats_df.to_excel(r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Statistics\voter_stats_FT.xlsx", index=False)

### Stats for each proposal

In [14]:
import os
import pandas as pd
import numpy as np

# Function to calculate Gini coefficient
def calculate_gini_coefficient(voting_power_list):
    sorted_voting_power = np.sort(voting_power_list)
    cumulative_sum = np.cumsum(sorted_voting_power)
    cumulative_proportion = cumulative_sum / np.sum(sorted_voting_power)
    n = len(sorted_voting_power)
    gini_coefficient = 1 - (2 * np.sum((n - np.arange(1, n + 1)) * cumulative_proportion) / (n * np.sum(sorted_voting_power)))
    return gini_coefficient

# Function to calculate the number of voters needed to achieve over 50%
def calculate_voters_needed_50_percent(sorted_voters):
    total_voting_power = sorted_voters['voting_power'].sum()
    cumulative_voting_power = 0
    voters_needed = 0

    for _, row in sorted_voters.iterrows():
        cumulative_voting_power += row['voting_power']
        voters_needed += 1

        if total_voting_power == 0 or (cumulative_voting_power / total_voting_power > 0.5):
            break

    return voters_needed

# Function to calculate HHI
def calculate_hhi(voting_power_list):
    total_voting_power = np.sum(voting_power_list)
    share_squared = np.sum((voting_power_list / total_voting_power) ** 2)
    return share_squared

# Function to calculate normalized HHI
def calculate_normalized_hhi(voting_power_list):
    n = len(voting_power_list)
    if n <= 1:  # Avoid division by zero or negative normalization for n=1
        return 0
    hhi = calculate_hhi(voting_power_list)
    normalized_hhi = (hhi - (1/n)) / (1 - (1/n))
    return normalized_hhi


def calculate_cr(sorted_voting_powers, top_n):
    if len(sorted_voting_powers) < top_n:
        # If there are fewer members than top_n, calculate CR for all available members
        top_n = len(sorted_voting_powers)
    return sum(sorted_voting_powers[:top_n]) / sum(sorted_voting_powers)

# Specify the directory containing the files
directory_path = r'D:\DAO_Voting_1101\merged_data_files\FT'
output_directory = r'D:\DAO_Voting_1101\Statistics\voter_stats\participation_rate&gini_coef&voters_needed_achieve50%_byproposal'
all_voters_cumulative = set()  # For cumulative participation rate across all proposals

for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        protocol_name = filename.split("_")[0].split(".")[0]
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
        df['voting_power'] = pd.to_numeric(df['voting_power'], errors='coerce')
        df = df.dropna(subset=['voting_power'])  # Drop NaN values first to avoid comparison with NaN
        df = df[df['voting_power'] > 0]  # Keep rows where 'voting_power' is greater than 0

        # Ensure proposals are processed in chronological order
        df = df.sort_values(by='start')
        
        results = []

        for proposal_id, group_df in df.groupby('proposal_id'):
            group_df = group_df.sort_values(by='voting_power', ascending=False)  # Sort by voting power for CR calculation
            current_voters = set(group_df['voter_address'])
            all_voters_cumulative.update(current_voters)
            
            # Metrics calculation
            gini_coefficient = calculate_gini_coefficient(group_df['voting_power'].values)
            voters_needed = calculate_voters_needed_50_percent(group_df)
            hhi = calculate_hhi(group_df['voting_power'].values)
            normalized_hhi = calculate_normalized_hhi(group_df['voting_power'].values)
            
            cr_top_1 = calculate_cr(group_df['voting_power'], 1)
            cr_top_3 = calculate_cr(group_df['voting_power'], 3)
            cr_top_5 = calculate_cr(group_df['voting_power'], 5)
            cr_top_7 = calculate_cr(group_df['voting_power'], 7)
            cr_top_10 = calculate_cr(group_df['voting_power'], 10)
            
            total_voters = len(current_voters)  # Size of the total voter set for each group
            simple_participation_rate = len(current_voters) / len(set(df['voter_address']))
            cumulative_participation_rate = len(all_voters_cumulative) / len(set(df['voter_address']))
            start_time = group_df['start'].iloc[0]

            results.append({
                'proposal_id': proposal_id,
                'simple_participation_rate': simple_participation_rate,
                'cumulative_participation_rate': cumulative_participation_rate,
                'gini_coefficient': gini_coefficient,
                'voters_needed_50_percent': voters_needed,
                'HHI': hhi,
                'normalized_HHI': normalized_hhi,
                'CR_top_1': cr_top_1,
                'CR_top_3': cr_top_3,
                'CR_top_5': cr_top_5,
                'CR_top_7': cr_top_7,
                'CR_top_10': cr_top_10,
                'total_voters': total_voters,
                'start_time': start_time,
                'start_real_time': pd.to_datetime(start_time, unit='s')
            })

        result_df = pd.DataFrame(results)
        output_filename = f'{protocol_name}_voter_stats.csv'
        output_path = os.path.join(output_directory, output_filename)
        result_df.to_csv(output_path, index=False)
        print(f"Saved voter stats for {protocol_name}")

Saved voter stats for 0xgov
Saved voter stats for 1inch
Saved voter stats for aave
Saved voter stats for aavegotchi
Saved voter stats for abachi
Saved voter stats for abracadabrabymerlinthemagician
Saved voter stats for alchemistcoin
Saved voter stats for alchemixstakers
Saved voter stats for alpacafinance
Saved voter stats for ampleforthorg
Saved voter stats for apecoin
Saved voter stats for apwine
Saved voter stats for arbitrumfoundation
Saved voter stats for babydogevote
Saved voter stats for badgerdao
Saved voter stats for balancer
Saved voter stats for bancornetwork
Saved voter stats for banklessvault
Saved voter stats for barnbridge
Saved voter stats for beanstalkdao
Saved voter stats for beanstalkfarms
Saved voter stats for beets
Saved voter stats for bestfork
Saved voter stats for bitdao
Saved voter stats for bitembassy
Saved voter stats for blockzerolabs
Saved voter stats for botto
Saved voter stats for cabindao
Saved voter stats for comp-vote
Saved voter stats for conic-dao
S

In [ ]:
# turnover rate

import os
import pandas as pd

# Specify the directory containing the files
directory_path = r"D:\DAO_Voting_1101\merged_data_files\FT"

# Output directory
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\consecutive_two_turnover_ratio"

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Sort DataFrame by 'start' column
        df.sort_values(by='end', inplace=True)

        # Turnover Rate Calculation
        proposals = df['proposal_id'].unique()

        turnover_ratios = {}
        for i in range(1, len(proposals)):
            current_proposal = proposals[i]
            previous_proposal = proposals[i - 1]

            current_voters = set(df[df['proposal_id'] == current_proposal]['voter_address'])
            previous_voters = set(df[df['proposal_id'] == previous_proposal]['voter_address'])

            intersection_count = len(current_voters.intersection(previous_voters))
            union_count = len(current_voters.union(previous_voters))

            turnover_ratio = intersection_count / union_count
            turnover_ratios[f'{previous_proposal} -> {current_proposal}'] = turnover_ratio
            
        # Create DataFrame from turnover_ratios
        result_df = pd.DataFrame(list(turnover_ratios.items()), columns=['proposal_pair','turnover_ratio'])

        # Extract protocol_name from the filename
        protocol_name = filename.split("_")[0].split(".")[0]

        # Save the DataFrame to CSV
        output_filename = f'{protocol_name}_turnover_ratio.csv'
        output_path = os.path.join(output_directory, output_filename)
        result_df.to_csv(output_path, index=False)

        print(f"Processed file: {filename}, Turnover Ratios saved to: {output_path}")


In [3]:
# 3 consecutive proposals as bundle, calculate turnover rate
import os
import pandas as pd

# Specify the directory containing the files
directory_path = r"D:\DAO_Voting_1101\merged_data_files\FT"

# Output directory for the new calculation
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\3_consecutive_bundle_turnover_ratio"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Sort DataFrame by 'end' column
        df.sort_values(by='end', inplace=True)

        # Turnover Rate Calculation for 3-consecutive proposal bundles
        proposals = df['proposal_id'].unique()

        turnover_ratios = {}
        # Start from the first full bundle
        for i in range(3, len(proposals)):
            # Define the bundles
            current_bundle = proposals[i-2:i+1]
            previous_bundle = proposals[i-3:i]

            # Collect voters for each bundle
            current_voters = set(df[df['proposal_id'].isin(current_bundle)]['voter_address'])
            previous_voters = set(df[df['proposal_id'].isin(previous_bundle)]['voter_address'])

            # Calculate intersection and union of voters
            intersection_count = len(current_voters.intersection(previous_voters))
            union_count = len(current_voters.union(previous_voters))

            turnover_ratio = intersection_count / union_count if union_count > 0 else 0
            turnover_ratios[f'{previous_bundle} -> {current_bundle}'] = turnover_ratio
            
        # Create DataFrame from turnover_ratios
        result_df = pd.DataFrame(list(turnover_ratios.items()), columns=['proposal_bundle_pair','turnover_ratio'])

        # Extract protocol_name from the filename
        protocol_name = filename.split("_")[0].split(".")[0]

        # Save the DataFrame to CSV
        output_filename = f'{protocol_name}_3_consecutive_bundle_turnover_ratio.csv'
        output_path = os.path.join(output_directory, output_filename)
        result_df.to_csv(output_path, index=False)

        print(f"Processed file: {filename}, 3-Consecutive Bundle Turnover Ratios saved to: {output_path}")


Processed file: 0xgov.eth_merged_data.csv, 3-Consecutive Bundle Turnover Ratios saved to: D:\DAO_Voting_1101\Statistics\voter_stats\3_consecutive_bundle_turnover_ratio\0xgov_3_consecutive_bundle_turnover_ratio.csv
Processed file: 1inch.eth_merged_data.csv, 3-Consecutive Bundle Turnover Ratios saved to: D:\DAO_Voting_1101\Statistics\voter_stats\3_consecutive_bundle_turnover_ratio\1inch_3_consecutive_bundle_turnover_ratio.csv
Processed file: aave.eth_merged_data.csv, 3-Consecutive Bundle Turnover Ratios saved to: D:\DAO_Voting_1101\Statistics\voter_stats\3_consecutive_bundle_turnover_ratio\aave_3_consecutive_bundle_turnover_ratio.csv
Processed file: aavegotchi.eth_merged_data.csv, 3-Consecutive Bundle Turnover Ratios saved to: D:\DAO_Voting_1101\Statistics\voter_stats\3_consecutive_bundle_turnover_ratio\aavegotchi_3_consecutive_bundle_turnover_ratio.csv
Processed file: abachi.eth_merged_data.csv, 3-Consecutive Bundle Turnover Ratios saved to: D:\DAO_Voting_1101\Statistics\voter_stats\3_c

In [6]:
folder_path = r"D:\DAO_Voting_1101\Statistics\voter_stats\consecutive_two_turnover_ratio"

# Iterate over all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Split the 'proposal_pair' column on the arrow symbol
        df['second_proposal'] = df['proposal_pair'].apply(lambda x: x.split(' -> ')[1])
        # Save the DataFrame back to the original file
        df.to_csv(file_path, index=False)

In [1]:
# over 15 per voter turnover ratio 
import os
import pandas as pd

# Specify the directory containing the files
directory_path = r"D:\DAO_Voting_1101\merged_data_files\FT"

# Output directory
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\consecutive_two_turnover_ratio\over15per_voter_turnover_ratio"

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Sort DataFrame by 'end' column
        df.sort_values(by='end', inplace=True)

        # Filter voters with more than 15% accountability
        df_filtered = df[df['accountability'] > 0.15]

        # Turnover Rate Calculation
        proposals = df_filtered['proposal_id'].unique()

        turnover_ratios = {}
        for i in range(1, len(proposals)):
            current_proposal = proposals[i]
            previous_proposal = proposals[i - 1]

            current_voters = set(df_filtered[df_filtered['proposal_id'] == current_proposal]['voter_address'])
            previous_voters = set(df_filtered[df_filtered['proposal_id'] == previous_proposal]['voter_address'])

            intersection_count = len(current_voters.intersection(previous_voters))
            union_count = len(current_voters.union(previous_voters))

            turnover_ratio = intersection_count / union_count if union_count != 0 else 0
            turnover_ratios[f'{previous_proposal} -> {current_proposal}'] = turnover_ratio
            
        # Create DataFrame from turnover_ratios
        result_df = pd.DataFrame(list(turnover_ratios.items()), columns=['proposal_pair','turnover_ratio'])

        # Extract protocol_name from the filename
        protocol_name = filename.split("_")[0].split(".")[0]

        # Save the DataFrame to CSV
        output_filename = f'{protocol_name}_over15per_voter_turnover_ratio.csv'
        output_path = os.path.join(output_directory, output_filename)
        result_df.to_csv(output_path, index=False)

print("Turnover Ratios saved to the output directory.")


Turnover Ratios saved to the output directory.


In [2]:
folder_path = r'D:\DAO_Voting_1101\Statistics\voter_stats\consecutive_two_turnover_ratio\over15per_voter_turnover_ratio'

# Iterate over all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Split the 'proposal_pair' column on the arrow symbol
        df['second_proposal'] = df['proposal_pair'].apply(lambda x: x.split(' -> ')[1])
        # Save the DataFrame back to the original file
        df.to_csv(file_path, index=False)


In [142]:
## check the column list

import os
import pandas as pd

# Specify the directory path
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\FT'

# Define the ideal column list
ideal_columns = [
    'proposal_id', 'voter_address', 'choice', 'voting_power', 'timestamp', 'title',
    'author', 'votes', 'type', 'choices', 'scores', 'Num_choices', 'Win_choice',
    'Win_score', 'Winrate', 'scores_total', 'discussion', 'link', 'quorum', 'start',
    'start_real_time', 'end', 'end_real_time', 'state', 'blockNumber',
    'relative_time_voting', 'win_option_index'
]

# Function to check if a DataFrame has the ideal columns
def has_ideal_columns(file_path):
    df = pd.read_csv(file_path, nrows=0)  # Read only the header
    return all(column in df.columns for column in ideal_columns)

# Check each file in the directory
non_fitted_files = [
    file for file in os.listdir(directory_path) if file.endswith('.csv') and not has_ideal_columns(os.path.join(directory_path, file))
]

# Print the non-fitted file names
if non_fitted_files:
    print("Non-fitted files:")
    for file in non_fitted_files:
        print(file)
else:
    print("All files have the ideal column list.")


All files have the ideal column list.


In [ ]:
import os
import pandas as pd
import numpy as np

# Specify the directory containing the files
directory_path = r'C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\merged_data_files\FT'

# Output directory
output_directory = r"C:\Users\ChunghyunHan\Desktop\DAO_Voting_1101\Statistics\voter_stats\avgRTV&top1_5_10&whale_votes_diff"

# Iterate through each file in the directory
for filename in os.listdir(directory_path)[2:]:
    try:
        if filename.endswith(".csv"):
            # Read the CSV file
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
            print(file_path)

            # Initialize an empty list to store the statistics for each proposal
            stats_list = []

            for proposal_id, group_df in df.groupby('proposal_id'):

                try:
                    # Calculate the specified statistics for each proposal
                    df['relative_time_voting'] = df['relative_time_voting'].astype(float)
                    
                    avg_RTV = group_df['relative_time_voting'].mean()
                    std_RTV = group_df['relative_time_voting'].std()

                    top_1_voter_address = group_df.loc[group_df['voting_power'].idxmax(), 'voter_address']
                    top_5_voters_address = group_df.nlargest(5, 'voting_power')['voter_address'].tolist()
                    top_10_voters_address = group_df.nlargest(10, 'voting_power')['voter_address'].tolist()

                    top_1_voter_VP = group_df.loc[group_df['voting_power'].idxmax(), 'voting_power']
                    top_5_voters_VP = group_df.nlargest(5, 'voting_power')['voting_power'].tolist()
                    top_10_voters_VP = group_df.nlargest(10, 'voting_power')['voting_power'].tolist()
                    
                    total_voting_power = group_df['voting_power'].sum()
                    top_1_voter_accountability = top_1_voter_VP / total_voting_power
                    top_5_voters_accountability = sum(top_5_voters_VP) / total_voting_power
                    top_10_voters_accountability = sum(top_10_voters_VP) / total_voting_power

                    top_1_voter_RTV = group_df.loc[group_df['voting_power'].idxmax(), 'relative_time_voting']
                    top_5_voters_RTV = group_df.nlargest(5, 'voting_power')['relative_time_voting'].tolist()
                    top_10_voters_RTV = group_df.nlargest(10, 'voting_power')['relative_time_voting'].tolist()

                    choice_top_1 = group_df.loc[group_df['voting_power'].idxmax(), 'choice']
                    choices_top_5 = group_df.nlargest(5, 'voting_power')['choice'].tolist()
                    choices_top_10 = group_df.nlargest(10, 'voting_power')['choice'].tolist()

                    choices_dict_top_5 = dict(zip(range(1, 6), choices_top_5))
                    choices_dict_top_10 = dict(zip(range(1, 11), choices_top_10))

                    # Calculate relative time voting average of voters other than top 10
                    other_voters_RTV = group_df[~group_df['voter_address'].isin(top_10_voters_address)]['relative_time_voting']
                    avg_other_voters_RTV = other_voters_RTV.mean()

                    choices = group_df['choices']
                    win_choice_index = group_df['win_option_index'].iloc[0]
                    
                    top1_winning_opt_difference = 1 if choice_top_1 != win_choice_index else 0

                    choices_identical_top_1_5 = {index: 0 if choices_dict_top_5.get(index, -1) == choice_top_1 else 1 for index in choices_dict_top_5}
                    choices_identical_top_1_10 = {index: 0 if choices_dict_top_10.get(index, -1) == choice_top_1 else 1 for index in choices_dict_top_10}

                    num_diff_top_1_5 = sum(choices_identical_top_1_5.values())
                    num_diff_top_1_10 = sum(choices_identical_top_1_10.values())

                    # Append the results to the list
                    result = {
                        'proposal_id': proposal_id,
                        'avg_RTV': avg_RTV,
                        'std_RTV': std_RTV,
                        'top_1_voter_address': top_1_voter_address,
                        'top_5_voters_address': top_5_voters_address,
                        'top_10_voters_address': top_10_voters_address,
                        'top_1_voter_RTV': top_1_voter_RTV,
                        'top_5_voters_RTV': top_5_voters_RTV,
                        'top_10_voters_RTV': top_10_voters_RTV,
                        'avg_other_voters_RTV': avg_other_voters_RTV,
                        'top_1_voter_accountability': top_1_voter_accountability,
                        'top_5_voters_accountability': top_5_voters_accountability,
                        'top_10_voters_accountability': top_10_voters_accountability,
                        'win_option_index' : win_choice_index,
                        'choice_top_1': choice_top_1,
                        'top1_win_option_diff': top1_winning_opt_difference,
                        'choice_top_5': choices_top_5,
                        'choice_top_10': choices_top_10,
                        'choices_identical_top_1_5': choices_identical_top_1_5,
                        'choices_identical_top_1_10': choices_identical_top_1_10,
                        'num_diff_top_1_5': num_diff_top_1_5,
                        'num_diff_top_1_10': num_diff_top_1_10
                    }

                    stats_list.append(result)
                except Exception as inner_error:
                    print(f"Error processing proposal_id {proposal_id} in file {filename}\nError: {str(inner_error)}")

            # Create a DataFrame from the list of statistics
            stats_df = pd.DataFrame(stats_list)

            # Extract protocol_name from the filename
            protocol_name = filename.split("_")[0].split(".")[0]

            # Save the DataFrame to CSV
            output_filename = f'{protocol_name}_top1_5_10_stats.csv'
            output_path = os.path.join(output_directory, output_filename)
            stats_df.to_csv(output_path, index=False)

            print(f"Processed file: {filename}")
    except Exception as e:
        print(f"Error processing file: {filename}\nError: {str(e)}")


In [ ]:


# holding and voting



In [ ]:
# (8) voters_holders

In [ ]:
import os
import pandas as pd

# Define the directory path
directory_path = r"D:\DAO_Voting_1101\merged_data_files\FT"



# Iterate through each CSV file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        protocol_name = filename.split("_")[0].split(".")[0]
        file_path = os.path.join(directory_path, filename)
        
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Identify voters who participated in more than one proposal
        multiple_participations = df.groupby('voter_address')['proposal_id'].nunique()
        multiple_participations = multiple_participations[multiple_participations > 1].index.tolist()

        # Filter the original DataFrame for voters with multiple participations
        filtered_df = df[df['voter_address'].isin(multiple_participations)]

        # Add a new column for protocol name in the first position
        filtered_df.insert(0, 'protocol_name', protocol_name)

        # Specify specific columns from the original DataFrame to include
        specific_columns = ['protocol_name', 'proposal_id', 'voter_address', 'choice', 'voting_power', 'author', 'choices', 'Win_choice', 'Winrate', 'start', 'end' ]

        # Concatenate the filtered DataFrame to the result DataFrame with specific columns
        result_df = pd.concat([result_df, filtered_df[specific_columns]])

    # Save the result DataFrame to a CSV file with protocol name in the filename
    result_df.to_csv(r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters\voting_results\{}_multiple_participations_voting_result.csv".format(protocol_name), index=False)
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()

    print(f"Processed file: {filename}")

In [ ]:
import os
import pandas as pd
# 1. multiple voters stat by protocol
## num max, min, avg, std of voters 

# Define the directory path
directory_path = r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters\voting_results"

# Empty list to store statistics for each file
stats_list = []

# Iterate through each CSV file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Group by 'voter_address' and aggregate the count and list of unique 'proposal_id'
        voter_participation_info = df.groupby('voter_address').agg({
            'proposal_id': ['count', lambda x: x.unique().tolist()],
            'author': lambda x: x.tolist(),
            'choice': lambda x: x.tolist()
        }).reset_index()

        # Add a new column 'unique_authors' with lists of unique authors
        voter_participation_info['unique_authors'] = df.groupby('voter_address')['author'].unique().apply(list).reset_index()['author']
        voter_participation_info['num_unique_authors'] = voter_participation_info['unique_authors'].apply(len)

        # Rename the columns for clarity
        voter_participation_info.columns = ['voter_address', 'num_proposals', 'proposal_ids', 'author', 'choice', 'unique_authors', 'num_unique_authors']

        # Convert the proposal_ids, author, and choice columns to strings for better representation
        voter_participation_info['proposal_ids'] = voter_participation_info['proposal_ids'].apply(list)
        voter_participation_info['author'] = voter_participation_info['author'].apply(list)
        voter_participation_info['choice'] = voter_participation_info['choice'].apply(list)
        voter_participation_info['unique_authors'] = voter_participation_info['unique_authors'].apply(list)

        # Export the result DataFrame to a CSV file
        protocol_name = filename.split("_")[0]
        voter_participation_info.to_csv(r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters\{}_multiple_time_voters.csv".format(protocol_name), index=False)

        # Update the stats list with information about the exported file
        stats_list.append({
            'protocol_name': protocol_name,
            'num_unique_multiple_voters': len(voter_participation_info),
            'num_proposals_min': voter_participation_info['num_proposals'].min(),
            'num_proposals_max': voter_participation_info['num_proposals'].max(),
            'num_proposals_avg': voter_participation_info['num_proposals'].mean(),
            'num_proposals_std': voter_participation_info['num_proposals'].std()
        })

        print(f"Processed file: {filename}")

# Create a DataFrame with summary statistics
summary_stats_df = pd.DataFrame(stats_list)

# Save the summary statistics DataFrame to a CSV file
summary_stats_df.to_csv(r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters\multiple_voters_summary_statistics.csv", index=False)

# Display the summary statistics DataFrame
print("\nSummary Statistics:")
print(summary_stats_df)


In [63]:
import os
import pandas as pd

def exploding( file_address ):
    df  = pd.read_csv(file_address)
    df['proposal_ids'] = df['proposal_ids'].map(lambda x: eval(x))
    df['author'] = df['author'].map(lambda x: eval(x))
    df['choice'] = df['choice'].map(lambda x: eval(x))
    df['pac'] = df.apply(lambda x:list(zip(*[x['proposal_ids'], x['author'], x['choice']])) , axis = 1)

    df_e = df.explode('pac').reset_index(drop = True)

    df_e['proposal_ids'] = df_e['pac'].map(lambda x: x[0])
    df_e['author'] = df_e['pac'].map(lambda x: x[1])
    df_e['choice'] = df_e['pac'].map(lambda x: x[2])
    
    return df_e

def find_reciprocal_relationship(df_e):
    df_c = df_e[(df_e['voter_address'].isin(df_e['author']) & df_e['voter_address'].isin(df_e['author']))] 

    df_c_condition1 = df_c[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1)
    df_c_condition2 = df_c[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1).value_counts()

    df_c2 = df_c[df_c_condition1.isin(df_c_condition2[df_c_condition2>1].index)].copy()

    df_c2['va_sorted'] = df_c2[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1)
    df_c2['same_as_sorted'] = df_c2['va_sorted'] == df_c2[['voter_address','author']].apply(lambda x: str([x['voter_address'],x['author']]),axis=1)

    reciprocal_ids = df_c2.groupby(['va_sorted'])['same_as_sorted'].unique()[df_c2.groupby(['va_sorted'])['same_as_sorted'].unique().map(len) > 1]
    
    relationship = df_c2[df_c2['va_sorted'].isin(reciprocal_ids.index)]

    return relationship
    

In [ ]:
# Define the directory path
directory_path = r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters"

# Output directory for reciprocal relationships
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\reciprocity"

# Iterate through each CSV file in the directory
for filename in sorted(os.listdir(directory_path)):
    
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)

        df_e = exploding(file_path)

        relationship = find_reciprocal_relationship(df_e)
        relationship_drop = relationship.drop(['unique_authors', 'num_unique_authors', 'pac', 'va_sorted', 'same_as_sorted'], axis=1)
        relationship_drop.to_csv(os.path.join(output_directory, f"{filename.split('.')[0].split('_')[0]}_reciprocal_relationships.csv"), index=False)

    print(f"Finished processing {filename}")

In [ ]:
# Empty list to store statistics for each file
stats_list = []

directory_path = r"D:\DAO_Voting_1101\Statistics\voter_stats\reciprocity"
# Iterate through each CSV file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Export the result DataFrame to a CSV file
        protocol_name = filename.split("_")[0]

        # Update the stats list with information about the exported file
        stats_list.append({
            'protocol_name': protocol_name,
            'num_reciprocal_proposals': len(df),
            'num_unique_authors': df['author'].nunique(),
            'num_unique_voters': df['voter_address'].nunique()
        })
    print(f"Processed file: {filename}")

# Create a DataFrame with summary statistics
summary_stats_df = pd.DataFrame(stats_list)

# Save the summary statistics DataFrame to a CSV file
summary_stats_df.to_csv(r"D:\DAO_Voting_1101\Statistics\voter_stats\reciprocal_summary_statistics.csv", index=False)


In [ ]:
# reciprocal voting 회수 : len

# unique author 수, unique voter 수 


# unique voter 중 whale voter 수
# proposal 별 whale voter list : {protocol}_top1_5_10_stats
    # 그 voter의 그 프로포절에서의 VP
    # 그 proposal의 토탈 voter 수, VP -> 그 가운데 해당 voter의 VP와 비중, 등수


# unique author 중 whale voter의 수 

## step 1. merged_data에서 해당 프로포절을 찾아서, 그 voter의 등수 찾기




In [72]:
import pandas as pd

def extract_proposals_and_rank(merged_data_path, reciprocal_path):
    # Read data from file 1
    df1 = pd.read_csv(merged_data_path, encoding='ISO-8859-1', low_memory=False)

    # Read data from file 2
    df2 = pd.read_csv(reciprocal_path)

    # Extract unique proposal_ids from file 
    unique_proposals = df2['proposal_ids'].unique()

    extracted_proposals = df1[df1['proposal_id'].isin(unique_proposals)].copy()

    # Calculate the rank of voters by 'voting_power' within each 'proposal_id' group
    extracted_proposals.loc[:, 'rank'] = extracted_proposals.groupby('proposal_id')['voting_power'].rank(ascending=False, method='min')

    return extracted_proposals


def extract_matching_voters(merged_data_path, reciprocal_path):
    # Read data from file 1
    extracted_file = extract_proposals_and_rank(merged_data_path, reciprocal_path)

    # Read data from file 2
    df2 = pd.read_csv(reciprocal_path)

    # Extract unique voter addresses from file 1
    unique_voters = df2['voter_address'].unique()
    
    # Extract rows from file 1 where 'voter_address' matches those in file 2
    matching_rows = extracted_file[extracted_file['voter_address'].isin(unique_voters)].copy()

    # Avoid SettingWithCopyWarning by using .loc to assign values
    matching_rows.loc[:, 'accountability'] = matching_rows['voting_power'] / matching_rows['scores_total']
    
    return matching_rows


In [ ]:
# unique voter 중 whale voter 수
# proposal 별 whale voter list : {protocol}_top1_5_10_stats
    # 그 voter의 그 프로포절에서의 VP
    # 그 proposal의 토탈 voter 수, VP -> 그 가운데 해당 voter의 VP와 비중, 등수

merged_data_path = r'D:\DAO_Voting_1101\merged_data_files\FT'
reciprocal_path = r'D:\DAO_Voting_1101\Statistics\voter_stats\reciprocity'
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\reciprocity\reciprocal_voting_data"

for filename in sorted(os.listdir(merged_data_path)):
    if filename.endswith(".csv"):
        m_protocol_name = filename.split("_")[0].split(".")[0]
        
        m_path = os.path.join(merged_data_path, filename)

        for r_filename in sorted(os.listdir(reciprocal_path)):

            if r_filename.split("_")[0] == m_protocol_name:
                
                r_path = os.path.join(reciprocal_path, r_filename)
                result = extract_matching_voters(m_path, r_path)
                result = result.drop(['title','votes','type','scores','Num_choices','Winrate','discussion','link','quorum','start','start_real_time','end','end_real_time','state','blockNumber'], axis=1)
                result.to_csv(os.path.join(output_directory, f"{m_protocol_name}_reciprocal_voting_data.csv"), index=False)

    print(f"Finished processing {filename}")


In [ ]:
## Reciprocity Stats
# proposal 개수 -> row 수 

# unique author 수, unique voter 수 
